In [1]:
import pandas as pd
import numpy as np
import cvxpy as cp
from scipy import sparse

import time 

import multiprocessing
from itertools import repeat


import matplotlib.pyplot as plt
%matplotlib inline


from pyomo.environ import *

### This notebook identifies the lower and upper bounds of the aggregate charging power and stored energy in EVs in San Francisco Bay area
- Calculate the results for the base case

In [3]:
q = sparse.load_npz('Parameters/Bay/pubCharger/q_rec.npz').toarray()[0]
demand_cum = sparse.load_npz(f"Parameters/Bay/pubCharger/demand_cum.npz").toarray()
Location = sparse.load_npz(f"Parameters/Bay/pubCharger/Location.npz").toarray()
N_e_total = len(q)
N_n = 9
N_t = 96
E_init = 0.5
E_terminal = 0.5
L = np.zeros((N_t, N_t))
for i in range(N_t):
    for j in range(N_t):
        if i >= j:
            L[i, j] = 1

N_e = 1000
groupNum = N_e_total // N_e
N_e_last = N_e_total - N_e * groupNum
groupShare_list = np.zeros(groupNum+1)
for g in range(groupNum+1):
    groupShare_list[g] = q[g*N_e:(g+1)*N_e].sum()/q.sum()


index_to_county = {
    0: 'San Francisco',
    1: 'San Mateo',
    2: 'Santa Clara',
    3: 'Alameda',
    4: 'Contra Costa',
    5: 'Solano',
    6: 'Napa',
    7: 'Sonoma',
    8: 'Marin'
}

In [3]:
boolIndex = {}
for countyInd in range(9):
    boolIndex[countyInd] = np.zeros((N_e_total, N_t))
    for t in range(N_t):
        boolIndex[countyInd][:, t] = Location[:, t] == countyInd
    

In [5]:
'''problem for each group'''

model = ConcreteModel()

# Variables
model.y_e_UB = Var(range(N_e), range(N_t), domain=NonNegativeReals, initialize=0)
model.y_e_LB = Var(range(N_e), range(N_t), domain=NonNegativeReals, initialize=0)
model.E_e_UB_omega = Var(range(N_e), range(N_t), domain=NonNegativeReals)
model.E_e_LB_omega = Var(range(N_e), range(N_t), domain=NonNegativeReals)
model.E_e_sum_weighted = Var(range(N_e), domain=NonNegativeReals, initialize=0)

model.y_capacity = Var(range(N_e), range(N_t), domain=NonNegativeReals, initialize=0)

model.ell_capacity = Param(range(N_n), domain=NonNegativeReals, mutable=True)
model.power_UB_para = Param(range(N_e), range(N_t), domain=NonNegativeReals, mutable=True)
model.demand_cum = Param(range(N_e), range(N_t), domain=NonNegativeReals, mutable=True)

# model.pene_para = Param(within=NonNegativeReals, mutable=True)
model.omega = Param(range(N_e), within=NonNegativeReals, mutable=True)
# model.q = Param(range(N_e), domain=NonNegativeReals, mutable=True)

# pene_para * q[e] * omega[e] 
model.coe_E = Param(range(N_e), domain=NonNegativeReals, mutable=True)

# model_last.pene_para * model_last.q[e] * model_last.omega_para[e] * boolIndex[countyInd][e, t]
model.coe_county0 = Param(range(N_e), range(N_t), domain=NonNegativeReals, mutable=True) 
model.coe_county1 = Param(range(N_e), range(N_t), domain=NonNegativeReals, mutable=True)
model.coe_county2 = Param(range(N_e), range(N_t), domain=NonNegativeReals, mutable=True)
model.coe_county3 = Param(range(N_e), range(N_t), domain=NonNegativeReals, mutable=True)
model.coe_county4 = Param(range(N_e), range(N_t), domain=NonNegativeReals, mutable=True)
model.coe_county5 = Param(range(N_e), range(N_t), domain=NonNegativeReals, mutable=True)
model.coe_county6 = Param(range(N_e), range(N_t), domain=NonNegativeReals, mutable=True)
model.coe_county7 = Param(range(N_e), range(N_t), domain=NonNegativeReals, mutable=True)
model.coe_county8 = Param(range(N_e), range(N_t), domain=NonNegativeReals, mutable=True)



# Constraints for individual trip chain
model.c1 = ConstraintList()
for e in range(N_e):
    for t in range(N_t):
        model.c1.add(expr=model.y_e_UB[e, t] <= model.y_capacity[e, t])
        model.c1.add(expr=model.y_e_LB[e, t] <= model.y_capacity[e, t])
        model.c1.add(expr=model.y_capacity[e, t] <= model.omega[e] * model.power_UB_para[e, t])
        model.c1.add(expr=model.E_e_LB_omega[e, t] <= model.E_e_UB_omega[e, t])
        model.c1.add(expr=model.omega[e] * E_init - model.omega[e] * model.demand_cum[e, t] + 
                    sum(L[t, tt] * model.y_e_UB[e, tt] * 0.25 for tt in range(N_t)) == model.E_e_UB_omega[e, t])
        model.c1.add(expr=model.omega[e] * E_init - model.omega[e] * model.demand_cum[e, t] + 
                    sum(L[t, tt] * model.y_e_LB[e, tt] * 0.25 for tt in range(N_t)) == model.E_e_LB_omega[e, t])
        model.c1.add(expr=model.E_e_LB_omega[e, t] >= 0)
        model.c1.add(expr=model.E_e_UB_omega[e, t] <= model.omega[e])
    model.c1.add(expr=model.E_e_LB_omega[e, N_t-1] >= model.omega[e] * E_terminal)
    model.c1.add(expr=model.E_e_sum_weighted[e] == model.coe_E[e] * sum(model.E_e_UB_omega[e, t]-model.E_e_LB_omega[e, t] for t in range(N_t)))

# Constraints for aggregate charging power capacity
model.c2 = ConstraintList()
for t in range(N_t):
    model.c2.add(sum(model.coe_county0[e, t] * model.y_capacity[e, t] for e in range(N_e)) <= model.ell_capacity[0])
    model.c2.add(sum(model.coe_county1[e, t] * model.y_capacity[e, t] for e in range(N_e)) <= model.ell_capacity[1])
    model.c2.add(sum(model.coe_county2[e, t] * model.y_capacity[e, t] for e in range(N_e)) <= model.ell_capacity[2])
    model.c2.add(sum(model.coe_county3[e, t] * model.y_capacity[e, t] for e in range(N_e)) <= model.ell_capacity[3])
    model.c2.add(sum(model.coe_county4[e, t] * model.y_capacity[e, t] for e in range(N_e)) <= model.ell_capacity[4])
    model.c2.add(sum(model.coe_county5[e, t] * model.y_capacity[e, t] for e in range(N_e)) <= model.ell_capacity[5])
    model.c2.add(sum(model.coe_county6[e, t] * model.y_capacity[e, t] for e in range(N_e)) <= model.ell_capacity[6])
    model.c2.add(sum(model.coe_county7[e, t] * model.y_capacity[e, t] for e in range(N_e)) <= model.ell_capacity[7])
    model.c2.add(sum(model.coe_county8[e, t] * model.y_capacity[e, t] for e in range(N_e)) <= model.ell_capacity[8])

# Objective
model.obj = Objective(expr=sum(model.E_e_sum_weighted[e] for e in range(N_e)), sense=maximize)

In [6]:
'''problem for each group'''

model_last = ConcreteModel()

# Variables
model_last.y_e_UB = Var(range(N_e_last), range(N_t), domain=NonNegativeReals, initialize=0)
model_last.y_e_LB = Var(range(N_e_last), range(N_t), domain=NonNegativeReals, initialize=0)
model_last.E_e_UB_omega = Var(range(N_e_last), range(N_t), domain=NonNegativeReals)
model_last.E_e_LB_omega = Var(range(N_e_last), range(N_t), domain=NonNegativeReals)
model_last.E_e_sum_weighted = Var(range(N_e_last), domain=NonNegativeReals, initialize=0)

model_last.y_capacity = Var(range(N_e_last), range(N_t), domain=NonNegativeReals, initialize=0)

model_last.ell_capacity = Param(range(N_n), domain=NonNegativeReals, mutable=True)
model_last.power_UB_para = Param(range(N_e_last), range(N_t), domain=NonNegativeReals, mutable=True)
model_last.demand_cum = Param(range(N_e_last), range(N_t), domain=NonNegativeReals, mutable=True)

# model_last.pene_para = Param(within=NonNegativeReals, mutable=True)
model_last.omega = Param(range(N_e_last), within=NonNegativeReals, mutable=True)
# model_last.q = Param(range(N_e_last), domain=NonNegativeReals, mutable=True)

# pene_para * q[e] * omega[e] 
model_last.coe_E = Param(range(N_e_last), domain=NonNegativeReals, mutable=True)

# model_last.pene_para * model_last.q[e] * model_last.omega_para[e] * boolIndex[countyInd][e, t]
model_last.coe_county0 = Param(range(N_e_last), range(N_t), domain=NonNegativeReals, mutable=True) 
model_last.coe_county1 = Param(range(N_e_last), range(N_t), domain=NonNegativeReals, mutable=True)
model_last.coe_county2 = Param(range(N_e_last), range(N_t), domain=NonNegativeReals, mutable=True)
model_last.coe_county3 = Param(range(N_e_last), range(N_t), domain=NonNegativeReals, mutable=True)
model_last.coe_county4 = Param(range(N_e_last), range(N_t), domain=NonNegativeReals, mutable=True)
model_last.coe_county5 = Param(range(N_e_last), range(N_t), domain=NonNegativeReals, mutable=True)
model_last.coe_county6 = Param(range(N_e_last), range(N_t), domain=NonNegativeReals, mutable=True)
model_last.coe_county7 = Param(range(N_e_last), range(N_t), domain=NonNegativeReals, mutable=True)
model_last.coe_county8 = Param(range(N_e_last), range(N_t), domain=NonNegativeReals, mutable=True)


# Constraints for individual trip chain
model_last.c1 = ConstraintList()
for e in range(N_e_last):
    for t in range(N_t):
        model_last.c1.add(expr=model_last.y_e_UB[e, t] <= model_last.y_capacity[e, t])
        model_last.c1.add(expr=model_last.y_e_LB[e, t] <= model_last.y_capacity[e, t])
        model_last.c1.add(expr=model_last.y_capacity[e, t] <= model_last.omega[e] * model_last.power_UB_para[e, t])
        model_last.c1.add(expr=model_last.E_e_LB_omega[e, t] <= model_last.E_e_UB_omega[e, t])
        model_last.c1.add(expr=model_last.omega[e] * E_init - model_last.omega[e] * model_last.demand_cum[e, t] + 
                    sum(L[t, tt] * model_last.y_e_UB[e, tt] * 0.25 for tt in range(N_t)) == model_last.E_e_UB_omega[e, t])
        model_last.c1.add(expr=model_last.omega[e] * E_init - model_last.omega[e] * model_last.demand_cum[e, t] + 
                    sum(L[t, tt] * model_last.y_e_LB[e, tt] * 0.25 for tt in range(N_t)) == model_last.E_e_LB_omega[e, t])
        model_last.c1.add(expr=model_last.E_e_LB_omega[e, t] >= 0)
        model_last.c1.add(expr=model_last.E_e_UB_omega[e, t] <= model_last.omega[e])
    model_last.c1.add(expr=model_last.E_e_LB_omega[e, N_t-1] >= model_last.omega[e] * E_terminal)
    model_last.c1.add(expr=model_last.E_e_sum_weighted[e] == model_last.coe_E[e] * sum(model_last.E_e_UB_omega[e, t]-model_last.E_e_LB_omega[e, t] for t in range(N_t)))

# Constraints for aggregate charging power capacity
model_last.c2 = ConstraintList()
for t in range(N_t):
    model_last.c2.add(sum(model_last.coe_county0[e, t] * model_last.y_capacity[e, t] for e in range(N_e_last)) <= model_last.ell_capacity[0])
    model_last.c2.add(sum(model_last.coe_county1[e, t] * model_last.y_capacity[e, t] for e in range(N_e_last)) <= model_last.ell_capacity[1])
    model_last.c2.add(sum(model_last.coe_county2[e, t] * model_last.y_capacity[e, t] for e in range(N_e_last)) <= model_last.ell_capacity[2])
    model_last.c2.add(sum(model_last.coe_county3[e, t] * model_last.y_capacity[e, t] for e in range(N_e_last)) <= model_last.ell_capacity[3])
    model_last.c2.add(sum(model_last.coe_county4[e, t] * model_last.y_capacity[e, t] for e in range(N_e_last)) <= model_last.ell_capacity[4])
    model_last.c2.add(sum(model_last.coe_county5[e, t] * model_last.y_capacity[e, t] for e in range(N_e_last)) <= model_last.ell_capacity[5])
    model_last.c2.add(sum(model_last.coe_county6[e, t] * model_last.y_capacity[e, t] for e in range(N_e_last)) <= model_last.ell_capacity[6])
    model_last.c2.add(sum(model_last.coe_county7[e, t] * model_last.y_capacity[e, t] for e in range(N_e_last)) <= model_last.ell_capacity[7])
    model_last.c2.add(sum(model_last.coe_county8[e, t] * model_last.y_capacity[e, t] for e in range(N_e_last)) <= model_last.ell_capacity[8])

# Objective
model_last.obj = Objective(expr=sum(model_last.E_e_sum_weighted[e] for e in range(N_e_last)), sense=maximize)


In [4]:
capacity_county_ori = pd.read_csv('geo_California/LoadCapacity_bay.csv', index_col = 0)
capacity_county = np.zeros(N_n)
for county_ind, county_name in index_to_county.items():
    capacity_temp = capacity_county_ori[capacity_county_ori['NAME'] == county_name]['LoadCapacity_MW'].values[0]
    capacity_county[county_ind] = capacity_temp
capacity_county *= 1000/56.5
capacityNum = 1
capacity_scale = [1]

peneList = [0.025]
peneNum = len(peneList)

ratio_pubCharger_list = [0.4]
ratioNum = len(ratio_pubCharger_list)

In [10]:
E_UB_rec = {}
E_LB_rec = {}
ell_UB_rec = {}


countyID_list = np.arange(9)

solver = SolverFactory('gurobi')
for i in range(peneNum):
    E_UB_rec[i] = {}
    E_LB_rec[i] = {}
    ell_UB_rec[i] = {}
    for ratio_pubCharger in ratio_pubCharger_list:
        E_UB_rec[i][ratio_pubCharger] = np.zeros((capacityNum, N_n*N_t))
        E_LB_rec[i][ratio_pubCharger] = np.zeros((capacityNum, N_n*N_t))
        ell_UB_rec[i][ratio_pubCharger] = np.zeros((capacityNum, N_n*N_t))
        p_max_total = sparse.load_npz(f"Parameters/Bay/pubCharger/p_max_ratio={ratio_pubCharger:.1f}.npz").toarray()
        power_UB_para_value = p_max_total
    

        for j in range(capacityNum):
            print (f"start: EV_pene={peneList[i]}, ratio={ratio_pubCharger}, scale={capacity_scale[j]}")
            omega_matrix = sparse.load_npz(
                f"Results/Bay/omega_ratio/omega_pene={peneList[i]:.3f}_ratio={ratio_pubCharger:.1f}.npz").toarray()
            capacity_value = capacity_county * capacity_scale[j] * (1+1e-5)
            pene_value = peneList[i]
            omega_value = omega_matrix[j]
            # groupShare = groupShare_list[g]
            coe_temp = pene_value * q
            model.pene_para = pene_value
            start_all = time.time()
            for g in range(groupNum):
                '''parameter seeting for each group'''
                for n in range(N_n):
                    model.ell_capacity[n] = capacity_value[n] * groupShare_list[g]

                for e in range(N_e):
                    model.coe_E[e] = pene_value * q[g * N_e + e]
                    model.omega[e] = omega_value[g * N_e + e]
                    for t in range(N_t):
                        model.power_UB_para[e, t] = power_UB_para_value[g * N_e + e, t]
                        model.demand_cum[e, t] = demand_cum[g * N_e + e, t]
                        model.coe_county0[e, t] = pene_value * q[g * N_e + e] * boolIndex[0][g * N_e + e, t]
                        model.coe_county1[e, t] = pene_value * q[g * N_e + e] * boolIndex[1][g * N_e + e, t]
                        model.coe_county2[e, t] = pene_value * q[g * N_e + e] * boolIndex[2][g * N_e + e, t]
                        model.coe_county3[e, t] = pene_value * q[g * N_e + e] * boolIndex[3][g * N_e + e, t]
                        model.coe_county4[e, t] = pene_value * q[g * N_e + e] * boolIndex[4][g * N_e + e, t]
                        model.coe_county5[e, t] = pene_value * q[g * N_e + e] * boolIndex[5][g * N_e + e, t]
                        model.coe_county6[e, t] = pene_value * q[g * N_e + e] * boolIndex[6][g * N_e + e, t]
                        model.coe_county7[e, t] = pene_value * q[g * N_e + e] * boolIndex[7][g * N_e + e, t]
                        model.coe_county8[e, t] = pene_value * q[g * N_e + e] * boolIndex[8][g * N_e + e, t]



                '''solve the problem for each group'''
                start = time.time()
                results = solver.solve(model, warmstart=True)

                if str(results['Solver'][0]['Termination condition']) == 'optimal':
                    y_UB_result = np.array(model.y_e_UB[:, :]()).reshape(N_e, N_t)
                    y_LB_result= np.array(model.y_e_LB[:, :]()).reshape(N_e, N_t)
                    ell_UB_result = np.array(model.y_capacity[:, :]()).reshape(N_e, N_t)


                    E_UB_group = np.zeros((N_n, N_t))
                    E_LB_group = np.zeros((N_n, N_t))
                    ell_UB_group = np.zeros((N_n, N_t))
                    for e in range(N_e):
                        E_UB_county_temp = np.zeros((N_n, N_t))
                        E_LB_county_temp = np.zeros((N_n, N_t))
                        ell_UB_temp = np.zeros((N_n, N_t))
                        for n in range(N_n):
                            countyID = countyID_list[n]
                            E_UB_county_temp[n] = boolIndex[countyID][g * N_e+e, :] * y_UB_result[e] * 0.25
                            E_LB_county_temp[n] = boolIndex[countyID][g * N_e+e, :] * y_LB_result[e] * 0.25
                            ell_UB_temp[n] = boolIndex[countyID][g * N_e+e, :] * ell_UB_result[e]

                        E_UB_county_temp = np.cumsum(E_UB_county_temp, axis = 1)
                        E_LB_county_temp = np.cumsum(E_LB_county_temp, axis = 1)

                        E_UB_group += E_UB_county_temp * coe_temp[g * N_e + e]
                        E_LB_group += E_LB_county_temp * coe_temp[g * N_e + e]
                        ell_UB_group += ell_UB_temp * coe_temp[g * N_e + e]

                    E_UB_rec[i][ratio_pubCharger][j] += E_UB_group.reshape(-1) * 56.5 / 1000
                    E_LB_rec[i][ratio_pubCharger][j] += E_LB_group.reshape(-1) * 56.5 / 1000
                    ell_UB_rec[i][ratio_pubCharger][j] += ell_UB_group.reshape(-1) * 56.5 / 1000

                    gap_E = (E_UB_rec[i][ratio_pubCharger][j].reshape(N_n, N_t) - 
                             E_LB_rec[i][ratio_pubCharger][j].reshape(N_n, N_t)).sum(axis = 0)
                    print (f"group={g}, obj={gap_E[-1]}, time={time.time()-start}")



            '''solve the problem for the last group'''
            model_last.pene_para = pene_value
            for n in range(N_n):
                model_last.ell_capacity[n] = capacity_value[n] * groupShare_list[groupNum]

            for e in range(N_e_last):
                # model_last.omega_para[e] = omega_value[groupNum * N_e + e]
                # model_last.q[e] = q[groupNum * N_e + e]
                model_last.coe_E[e] = pene_value * q[groupNum * N_e + e]
                model_last.omega[e] = omega_value[groupNum * N_e + e]
                for t in range(N_t):
                    model_last.power_UB_para[e, t] = power_UB_para_value[groupNum * N_e + e, t]
                    model_last.demand_cum[e, t] = demand_cum[groupNum * N_e + e, t]
                    model_last.coe_county0[e, t] = pene_value * q[groupNum * N_e + e] * boolIndex[0][groupNum * N_e + e, t]
                    model_last.coe_county1[e, t] = pene_value * q[groupNum * N_e + e] * boolIndex[1][groupNum * N_e + e, t]
                    model_last.coe_county2[e, t] = pene_value * q[groupNum * N_e + e] * boolIndex[2][groupNum * N_e + e, t]
                    model_last.coe_county3[e, t] = pene_value * q[groupNum * N_e + e] * boolIndex[3][groupNum * N_e + e, t]
                    model_last.coe_county4[e, t] = pene_value * q[groupNum * N_e + e] * boolIndex[4][groupNum * N_e + e, t]
                    model_last.coe_county5[e, t] = pene_value * q[groupNum * N_e + e] * boolIndex[5][groupNum * N_e + e, t]
                    model_last.coe_county6[e, t] = pene_value * q[groupNum * N_e + e] * boolIndex[6][groupNum * N_e + e, t]
                    model_last.coe_county7[e, t] = pene_value * q[groupNum * N_e + e] * boolIndex[7][groupNum * N_e + e, t]
                    model_last.coe_county8[e, t] = pene_value * q[groupNum * N_e + e] * boolIndex[8][groupNum * N_e + e, t]

            results_last = solver.solve(model_last)

            if str(results_last['Solver'][0]['Termination condition']) == 'optimal':
                y_UB_result = np.array(model_last.y_e_UB[:, :]()).reshape(N_e_last, N_t)
                y_LB_result= np.array(model_last.y_e_LB[:, :]()).reshape(N_e_last, N_t)
                ell_UB_result = np.array(model_last.y_capacity[:, :]()).reshape(N_e_last, N_t)


                E_UB_group = np.zeros((N_n, N_t))
                E_LB_group = np.zeros((N_n, N_t))
                ell_UB_group = np.zeros((N_n, N_t))
                for e in range(N_e_last):
                    E_UB_county_temp = np.zeros((N_n, N_t))
                    E_LB_county_temp = np.zeros((N_n, N_t))
                    ell_UB_temp = np.zeros((N_n, N_t))
                    for n in range(N_n):
                        countyID = countyID_list[n]
                        E_UB_county_temp[n] = boolIndex[countyID][groupNum * N_e+e, :] * y_UB_result[e] * 0.25
                        E_LB_county_temp[n] = boolIndex[countyID][groupNum * N_e+e, :] * y_LB_result[e] * 0.25
                        ell_UB_temp[n] = boolIndex[countyID][groupNum * N_e+e, :] * ell_UB_result[e]

                    E_UB_county_temp = np.cumsum(E_UB_county_temp, axis = 1)
                    E_LB_county_temp = np.cumsum(E_LB_county_temp, axis = 1)

                    E_UB_group += E_UB_county_temp * coe_temp[groupNum * N_e + e]
                    E_LB_group += E_LB_county_temp * coe_temp[groupNum * N_e + e]
                    ell_UB_group += ell_UB_temp * coe_temp[groupNum * N_e + e]

                E_UB_rec[i][ratio_pubCharger][j] += E_UB_group.reshape(-1) * 56.5 / 1000
                E_LB_rec[i][ratio_pubCharger][j] += E_LB_group.reshape(-1) * 56.5 / 1000
                ell_UB_rec[i][ratio_pubCharger][j] += ell_UB_group.reshape(-1) * 56.5 / 1000

                gap_E_last = (E_UB_rec[i][ratio_pubCharger][j].reshape(N_n, N_t) - 
                              E_LB_rec[i][ratio_pubCharger][j].reshape(N_n, N_t)).sum(axis = 0)
                print(f"ratio={ratio_pubCharger}, scale={capacity_scale[j]}," \
                       f"pene={pene_value}, obj={gap_E_last[-1]}, time={time.time()-start_all}")
            else:
                print(f"ratio={ratio_pubCharger}, scale={capacity_scale[j]}, pene={pene_value}, infeasible")

            # break # only one scale
        break # only one ratio
    break   # only one pene
                

        

start: EV_pene=0.025, ratio=0.4, scale=0.5
group=0, obj=147.72209446728726, time=26.215078830718994
group=1, obj=278.77177462855406, time=25.846982717514038
group=2, obj=407.47152148289166, time=25.10426688194275
group=3, obj=523.500455800942, time=25.373743057250977
group=4, obj=630.9044482747503, time=27.28295397758484
group=5, obj=742.575165398064, time=26.065353870391846
group=6, obj=853.8731850125197, time=26.067677974700928
group=7, obj=978.126565399639, time=31.17904496192932
group=8, obj=1086.5823711475555, time=32.930097818374634
group=9, obj=1184.0191012989615, time=29.418995141983032
group=10, obj=1235.3915119212631, time=20.944390058517456
group=11, obj=1281.3044647796646, time=20.999558925628662
group=12, obj=1330.3597990922044, time=20.975252151489258
group=13, obj=1372.5372962831277, time=20.999995708465576
group=14, obj=1410.4820150920928, time=21.12121081352234
group=15, obj=1456.2530106419918, time=21.321127891540527
group=16, obj=1504.1297391607, time=21.173424005508

In [ ]:
import os
filePath = 'Results/Bay/AggregateModel_realPene'


for i in range(peneNum):
    pene_value = float(peneList[i])
    path_temp = filePath + f'/pene={pene_value:.3f}'
    isExists=os.path.exists(path_temp) #判断路径是否存在，存在则返回true 
    isExists=os.path.exists(path_temp) #判断路径是否存在，存在则返回true
    if not isExists:
        os.makedirs(path_temp)  

    for ratio_pubCharger in ratio_pubCharger_list:
        for j in range(capacityNum):
            """save the energy bounds"""
            E_UB_sparse = sparse.csr_matrix(E_UB_rec[i][ratio_pubCharger][j])
            E_LB_sparse = sparse.csr_matrix(E_LB_rec[i][ratio_pubCharger][j])
            sparse.save_npz(f"{path_temp}/E_UB_ratio={ratio_pubCharger:.1f}_scale={capacity_scale[j]:.1f}.npz", E_UB_sparse)
            sparse.save_npz(f"{path_temp}/E_LB_ratio={ratio_pubCharger:.1f}_scale={capacity_scale[j]:.1f}.npz", E_LB_sparse)
            print (f"pene={peneList[i]:.3f}, ratio={ratio_pubCharger:.1f}, scale={capacity_scale[j]:.1f}, saved")
            break
        break
    break


pene=0.025, ratio=0.4, scale=0.5, saved
pene=0.025, ratio=0.4, scale=1.0, saved


In [5]:
p_max_total = sparse.load_npz(f"Parameters/Bay/pubCharger/p_max_ratio={0.4}.npz").toarray()
capacity_county_MW = capacity_county * 56.5/1000
capacity_realize_MW = np.zeros((N_n, N_t))
for n in range(N_n):
    for t in range(N_t):
        capacity_realize_MW[n] = np.minimum((p_max_total * boolIndex[n]).T @ (q * 0.025) * 56.5/1000, capacity_county_MW[n])

# ell_UB_new_sparse = sparse.csr_matrix(capacity_realize_MW.reshape(-1))
# path_temp = 'Results/Bay/AggregateModel_realPene/pene=0.025'
# sparse.save_npz(f"{path_temp}/ell_UB_new_ratio={0.4}_scale={1.0}.npz", ell_UB_new_sparse)